In [11]:
import json
import os

import numpy as np
import pandas as pd

from tkinter import *
from tkinter import messagebox

In [12]:
class Utils:
    def __init__(self):
        pass
    
    @staticmethod
    def size_to_geometry(window_width, window_height, window_x, window_y):
        size_geometry = str(window_width) + 'x' + str(window_height)
        position_geometry = str(window_x) + '+' + str(window_y)
        geometry = size_geometry + '+' + position_geometry
        return geometry
        
    @staticmethod
    def create_window(title, geometry, is_resizable=False):
        window = Tk()
        window.geometry(geometry)
        window.title(title)
        window.resizable(width=is_resizable, height=is_resizable)
        return window
    
    @staticmethod
    def create_canvas(window, width, height, x=0, y=0, bg='default', highlight=0):
        if bg=='default':
            bg = window.cget('bg')
            
        canvas = Canvas(window, width=width, height=height, highlightthickness=highlight, bg=bg)
        canvas.place(x=x, y=y)
        return canvas
    
    @staticmethod
    def create_btn(window, width, height, x=0, y=0, bg='default', default_text='', font='Courier', font_size=20, command=None):
        if bg=='default':
            bg = window.cget('bg')
            
        button = Button(window, width=width, height=height, bg=bg, text=default_text, command=command)
        button.configure(font=(font,font_size))
        button.place(x=x, y=y)
        return button
    
    @staticmethod
    def create_label(window, x=0, y=0, bg='default', default_text='', font='Courier', font_size=14):
        if bg=='default':
            bg = window.cget('bg')
            
        label = Label(window, text=default_text, bg=bg, font=(font, font_size))
        label.place(x=x, y=y)
        return label
    
    @staticmethod
    def create_textbox(window, width, height, x=0, y=0, default_text='', font='Courier', font_size=14):
        textbox = Text(window, width=width, height=height, font=(font, font_size))
        textbox.place(x=x, y=y)
        textbox.insert(1.0, default_text)
        return textbox
    
    @staticmethod
    def get_textbox(textbox, to_num=False):
        txt = textbox.get('1.0', 'end').replace('\n', '')
        
        if to_num: return int(txt)
        else: return txt
        
    @staticmethod
    def set_textbox(textbox, txt=''):
        textbox.delete('1.0', 'end')
        textbox.insert(1.0, txt)
        
    @staticmethod
    def json_to_array(json_file, prints=False):
        dic = json.loads(json_file)
        arr = []
        for i in range(20):
            arr.append([])
            for j in range(20):
                arr[i].append(dic[i][j]['value'])
        
        if prints:
            for i in range(20):
                print(arr[i])
            print('')
        
        return arr
    

In [13]:
class GridModule:
    def __init__(self, window, parent, grid_width, grid_height, grid_x, grid_y, default_size=20):
        self.window = window
        self.parent = parent
        
        self.default_size = default_size
        
        self.frame_size = {'w':grid_width, 'h':grid_height}
        self.frame_position = {'x':grid_x, 'y':grid_y}
        self.grid_count = {'x':self.default_size, 'y':self.default_size}
        self.grid_size = {'w':self.frame_size['w']/self.grid_count['x'], 'h':self.frame_size['h']/self.grid_count['y']}
        self.padding = {'canvas':20, 'grid':5}
        
        self.make_grid()
        
    def get_config(self):
        data = { 'size':self.frame_size, 'position':self.frame_position, 'grid_count':self.grid_count, 'grid_size':self.grid_size }
        return data
    
    def make_grid(self, grid_color='white'):
        self.canvas = Utils.create_canvas(self.window, width=self.frame_size['w'] + self.padding['canvas'], 
                                          height=self.frame_size['h'] + self.padding['canvas'],
                                          x=self.frame_position['x'], y=self.frame_position['y'])
        
        for i in range(self.grid_count['x']):
            for j in range(self.grid_count['y']):
                self.canvas.create_rectangle(i*self.grid_size['w']+self.padding['grid'], j*self.grid_size['h']+self.padding['grid'],
                                           (i+1)*self.grid_size['w']+self.padding['grid'],
                                           (j+1)*self.grid_size['h']+self.padding['grid'],fill=grid_color)

In [14]:
class CreateModule:
    def __init__(self, window, parent, create_height, create_width, create_x, create_y, bg='cyan'):
        self.window = window
        self.parent = parent
        self.canvas = Utils.create_canvas(window, create_width, create_height, create_x, create_y, bg)
        
        Utils.create_label(self.canvas, x=30, y=150, default_text='Size', font_size=20)
        Utils.create_label(self.canvas, x=150, y=150, default_text='X', font_size=20)
        
        self.x_textbox = Utils.create_textbox(self.canvas, width=2, height=1, x=110, y=150, default_text='1', font_size=20)
        self.y_textbox = Utils.create_textbox(self.canvas, width=2, height=1, x=175, y=150, default_text='1', font_size=20)
        
        create_btn = Utils.create_btn(self.canvas, width=19, height=1, x=20, y=0, bg='cyan', default_text='Create',
                                     font_size=20, command=self.create_furniture)
        
    def create_furniture(self):
        size_w = Utils.get_textbox(self.x_textbox, to_num=True)
        size_h = Utils.get_textbox(self.y_textbox, to_num=True)
        
        grid_module = self.parent.grid_module
        grid_data = grid_module.get_config()
        
        frame_position = grid_data['position']
        frame_size = grid_data['size']
        grid_count = grid_data['grid_count']
        grid_size = grid_data['grid_size']
        
        if size_w <= 0:
            size_w = 1
            Utils.set_textbox(self.x_textbox, txt=str(1))
        elif size_w > grid_count['x']:
            size_w = grid_count['x']
            Utils.set_textbox(self.x_textbox, txt=str(size_w))
            
        if size_h <= 0:
            size_h = 1
            Utils.set_textbox(self.y_textbox, txt=str(1))
        elif size_h > grid_count['y']:
            size_h = grid_count['y']
            Utils.set_textbox(self.y_textbox, txt=str(size_h))
            
        xlist = [x+frame_position['x'] + 5 for x in range(0, frame_size['w'], int(grid_size['w']))]
        ylist = [y+frame_position['y'] + 5 for y in range(0, frame_size['h'], int(grid_size['h']))]
        
        obj_size = {'w':size_w * grid_size['w'] - 1, 'h':size_h * grid_size['h'] - 1}
        init_pos = {'x':frame_position['x'] - grid_size['w'], 'y':frame_position['y'] - grid_size['h']}
        can_pos = {'xlist':xlist, 'ylist':ylist}
        
        furniture = Furniture(self.window, self.parent, size=obj_size, init_pos=init_pos, can_pos=can_pos, bg='blue')
        
        self.parent.furniture_list.append(furniture)
        self.parent.focus = furniture

In [15]:
class ListModule():
    def __init__(self, window, parent, list_width, list_height, list_x, list_y):
        self.window = window
        self.parent = parent
        self.canvas = Utils.create_canvas(window, list_width, list_height, list_x, list_y)
        
        self.frame = Frame(window)
        self.frame.grid(row=0, column=0, sticky='n')
        
        self.furniture_list = pd.read_csv('data/color.csv')
        
        rows=8;columns=2;n=0
        for i in range(0, rows):
            for j in range(0, columns):
                self.make_button(canvas=self.canvas, n=n, row=i, column=j)
                n+=1
                
        self.frame.grid_columnconfigure(1, minsize=20)
        
    def make_button(self, canvas, n, row, column):
        text = self.furniture_list['Type'][n]
        bg = self.furniture_list['Color'][n]
        active = self.furniture_list['ActiveColor'][n]
        font_color = self.furniture_list['TextColor'][n]
        
        btn = Button(canvas, text=text, command=lambda:self.click(btn), width=15, height=4)
        btn.config(font=('Courier', 14), bg=bg, activebackground=active, fg=font_color, activeforeground=font_color)
        btn.grid(row=row, column=column, sticky='we')
        
    def click(self, button):
        if self.parent.focus:
            focus = self.parent.focus
            focus_object = focus.get_object()
            
            text = button.cget('text')
            color = button.cget('bg')
            font_color = button.cget('fg')
            focus.org_color = color
            focus.org_text_color = font_color
            
            focus_object.config(bg=color)
            focus_object.itemconfig(focus_object.find_withtag(focus.text), text=text, fill=font_color, font=('Courier', 10))
            self.parent.focus = None

In [16]:
class Furniture:
    def __init__(self, window, parent, size, init_pos, can_pos, bg='cyan'):
        self.window = window
        self.parent = parent
        
        self.position = {'x':init_pos['x'], 'y':init_pos['y']}
        self.size = {'w':size['w'], 'h':size['h']}
        self.can_pos = can_pos
        
        self.obj = Utils.create_canvas(window, width=self.size['w'], height=self.size['h'],
                                      x=self.position['x'], y=self.position['y'], bg=bg)
        self.obj.bind('<Button-1>', self.click)
        self.obj.bind('<B1-Motion>', self.move)
        self.obj.bind('<ButtonRelease-1>', self.release)
        
        self.org_color = 'blue'
        self.org_text_color = 'black'
        self.text_pos = {'x':self.size['w']/2, 'y':self.size['h']/2}
        self.text = self.obj.create_text(self.text_pos['x'], self.text_pos['y'], fill='black', text='')
        
    def get_object(self):
        return self.obj
    
    def get_config(self):
        data = {
            'position': self.position,
            'size': self.size,
            'type':self.obj.itemcget(self.text, 'text')
        }
        return data
    
    def update_position(self):
        self.obj.place(x=self.position['x'], y=self.position['y'])
        
    def click(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        self.diff_x = int(mouse_x) - self.position['x']
        self.diff_y = int(mouse_y) - self.position['y']
        
        for furniture in self.parent.furniture_list:
            furniture.get_object().config(bg=furniture.org_color)
        self.parent.focus = self
        self.obj.itemconfig(self.obj.find_withtag(self.text), fill='white')
        self.obj.config(bg='purple')
        
    def move(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        self.position['x'] = int(mouse_x) - self.diff_x
        self.position['y'] = int(mouse_y) - self.diff_y
        self.update_position()
        
    def release(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        grid_data = self.parent.grid_module.get_config()
        minx = grid_data['position']['x']
        miny = grid_data['position']['y']
        maxx = grid_data['position']['x'] + grid_data['size']['w']
        maxy = grid_data['position']['y'] + grid_data['size']['h']
        
        if self.is_over_grid(mouse_x, mouse_y, minx, miny, maxx, maxy):
            return
        else:
            if mouse_x < minx and mouse_y < miny:
                self.position['x'] = minx + 6
                self.position['y'] = miny + 6
            elif mouse_x > maxx and mouse_y > maxy:
                self.position['x'] = maxx - self.size['w'] + 5
                self.position['y'] = maxy - self.size['h'] + 5
            elif mouse_x < minx:
                self.position['x'] = minx + 6
            elif mouse_x > maxx:
                self.position['x'] = maxx - self.size['w'] + 5
            elif mouse_y < miny:
                self.position['y'] = miny + 6
            elif mouse_y > maxy:
                self.position['y'] = maxy - self.size['h'] + 5
                
            if mouse_x >= minx and mouse_x <= maxx:
                place_x = self.can_pos['xlist'][0]
                for x  in self.can_pos['xlist']:
                    if self.position['x'] > x:
                        place_x = x
                self.position['x'] = place_x + 1
                
            if mouse_y >= miny and mouse_y <= maxy:
                place_y = self.can_pos['ylist'][0]
                for y in self.can_pos['ylist']:
                    if self.position['y'] > y:
                        place_y = y
                self.position['y'] = place_y + 1
        self.update_position()
            
    def is_over_grid(self, mouse_x, mouse_y, minx, miny, maxx, maxy):
        if mouse_x < minx - self.size['w']/2 or mouse_x > maxx + self.size['w']/2 or mouse_y < miny - self.size['h']/2 or mouse_y > maxy + self.size['h']/2:
            return True
        return False


In [40]:
class Application:
    def __init__(self, window_width, window_height, window_x, window_y, 
                 title='Furniture Layout Batch Application', is_resizable=False):
        
        geometry = Utils.size_to_geometry(window_width, window_height, window_x, window_y)
        self.window = Utils.create_window(title, geometry, is_resizable)
        
        self.grid_module = GridModule(self.window, self, grid_width=600, grid_height=600, grid_x=450, grid_y=200)
        self.create_module = CreateModule(self.window, self, create_width=350, create_height=800, create_x=1100, create_y=80)
        self.list_module = ListModule(self.window, self, list_width=350, list_height=800, list_x=50, list_y=80)
        
        self.create_title()
            
        self.reset_btn = Utils.create_btn(self.window, width=10, height=1, x=500, y=850, bg='cyan', default_text='Reset',
                                         command = self.clear_object)
        self.submit_btn = Utils.create_btn(self.window, width=10, height=1, x=800, y=850, bg='cyan', default_text='Submit',
                                         command = self.submit)
        
        self.furniture_list = []
        self.focus = None
        
        Utils.create_label(self.window, x=500, y=100, default_text='Name', font_size=20)
        self.name_textbox = Utils.create_textbox(self.window, width=10, height=1, x=500, y=150, default_text='', font_size=20)
        
        Utils.create_label(self.window, x=850, y=100, default_text='Type', font_size=20)
        self.type_textbox = Utils.create_textbox(self.window, width=10, height=1, x=850, y=150, default_text='', font_size=20)
        
    def runApp(self):
        self.window.mainloop()
        
    def create_title(self):
        Utils.create_label(self.window, default_text='RoomSize', font_size=30, x=650, y=25)
        Utils.create_label(self.window, default_text='20 X 20', font_size=30, x=660, y=75)
        
    def clear_object(self):
        for furniture in self.furniture_list:
            furniture.get_object().destroy()
        self.furniture_list = []
        self.focus = None
        self.create_grid()
    
    def create_grid(self):
        self.grids = []
        for i in range(20):
            self.grids.append([])
            for j in range(20):
                self.grids[i].append({'value':0, 'type':''})
    
    def to_json(self):
        self.name = Utils.get_textbox(self.name_textbox)
        self.room_type = Utils.get_textbox(self.type_textbox)
        
        self.data = {'name':self.name, 'type':self.room_type, 'data':self.grids}
        json_data = json.dumps(self.data)
        
        file_name = 'data/ikea_' + self.data['name'] + '.json'
        with open(file_name, 'w') as json_file:
            json.dump(json_data, json_file)
            
        self.window.destroy()
    
    def submit(self):
        self.create_grid()
        for furniture in self.furniture_list:
            furniture_data = furniture.get_config()
            x = int((furniture_data['position']['x']-450)//30)
            y = int((furniture_data['position']['y']-200)//30)
            w = int((furniture_data['size']['w']-20)//30) + 1
            h = int((furniture_data['size']['h']-20)//30) + 1
            
            for i in range(y, y+h):
                for j in range(x, x+w):
                    if self.grids[i][j]['value'] == 1:                        
                        messagebox.showerror('Error', 'Duplicate Object')
                        return
            
            for i in range(y, y+h):
                for j in range(x, x+w):
                    self.grids[i][j]['value'] = 1
                    self.grids[i][j]['type'] = furniture_data['type']

        self.to_json()

In [41]:
app = Application(window_width=1500, window_height=1000, window_x=200, window_y=200)
app.runApp()